In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
from sklearn.dummy import DummyClassifier
from ConstantClassifier import ConstantClassifier
import configs
import warnings
import classifier_utils
import seaborn as sns
warnings.filterwarnings("ignore")

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict, GridSearchCV, validation_curve
from sklearn.impute import SimpleImputer
import math

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])
non_features_columns.extend(["has_branch_merge_message_indicator"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [4]:
df_test = pd.read_csv("../../data/dataset-test.csv")
target_names = sorted(list(df_test['developerdecision'].unique()))
df_na = df_test[df_test.isna().any(axis=1)]

len(df_na) / len(df_test)

0.18596892574587742

# Class distributions

### Normalized (%)

In [5]:
import importlib
importlib.reload(classifier_utils)
class_distribution_normalized = classifier_utils.get_projects_class_distribution(projects, include_overall=True, training=False)

class_distribution_normalized

,Project,Version 1,Version 2,Combination,ConcatenationV1V2,ConcatenationV2V1,Manual,None
0,Ramblurr__Anki-Android,44.10,27.18,8.21,0.51,0.51,19.49,0.00
1,apache__directory-server,44.91,53.29,0.00,0.00,0.00,1.80,0.00
2,android__platform_frameworks_base,66.05,8.95,8.61,3.72,1.18,11.15,0.17
3,freenet__fred,43.08,17.79,20.16,7.91,0.40,10.28,0.40
4,alexo__wro4j,32.46,11.30,27.83,1.45,0.00,25.80,1.16
5,apache__lucene-solr,26.72,22.27,18.22,7.29,1.21,21.05,3.24
6,getrailo__railo,27.87,39.34,9.84,9.02,0.00,13.11,0.82
7,atlasapi__atlas,20.00,46.67,10.77,5.64,2.05,13.85,1.03
8,hibernate__hibernate-orm,23.53,18.82,35.29,6.47,0.00,14.12,1.76
9,CloudStack-extras__CloudStack-archive,42.80,18.18,7.95,18.18,1.14,11.36,0.38


### Count

In [6]:
class_distribution_count = classifier_utils.get_projects_class_distribution(projects, False, include_overall=True, training=False)

class_distribution_count

,Project,Version 1,Version 2,Combination,ConcatenationV1V2,ConcatenationV2V1,Manual,None
0,Ramblurr__Anki-Android,86,53,16,1,1,38,0
1,apache__directory-server,75,89,0,0,0,3,0
2,android__platform_frameworks_base,391,53,51,22,7,66,1
3,freenet__fred,109,45,51,20,1,26,1
4,alexo__wro4j,112,39,96,5,0,89,4
5,apache__lucene-solr,66,55,45,18,3,52,8
6,getrailo__railo,34,48,12,11,0,16,1
7,atlasapi__atlas,39,91,21,11,4,27,2
8,hibernate__hibernate-orm,40,32,60,11,0,24,3
9,CloudStack-extras__CloudStack-archive,113,48,21,48,3,30,1


# Classification results

In [7]:
model = RandomForestClassifier(random_state=99, n_jobs=-2, n_estimators=400, max_features=0.3, min_samples_leaf=1)

### Training/validation validation data

Uses 80% of the data with a 10-fold cross-validation to calculate the average accuracy for each project

In [8]:
results = classifier_utils.ProjectsResults(model, projects, non_features_columns)

In [9]:
report = results.get_report_df(include_overall=True)
report

,project,observations,observations (wt NaN),precision,recall,f1-score,accuracy,baseline (majority),improvement
0,CCI-MIT__XCoLab,5512,3757,0.9720,0.97600,0.974,0.97600,0.57300,0.9430
1,apache__directory-server,845,652,0.9380,0.94200,0.940,0.94200,0.51200,0.8810
2,jgralab__jgralab,2072,1802,0.8640,0.86700,0.864,0.86700,0.49100,0.7380
3,CloudStack-extras__CloudStack-archive,1424,1106,0.7990,0.80600,0.800,0.80600,0.43700,0.6550
4,Unidata__thredds,1154,950,0.9120,0.91800,0.913,0.91800,0.77700,0.6320
5,apache__accumulo,4113,3148,0.8560,0.86200,0.858,0.86200,0.63500,0.6210
6,Ramblurr__Anki-Android,892,759,0.7170,0.73900,0.720,0.73900,0.43900,0.5350
7,getrailo__railo,815,572,0.6960,0.71000,0.701,0.71000,0.37800,0.5340
8,apache__lucene-solr,1256,974,0.6390,0.64600,0.639,0.64600,0.26600,0.5170
9,TeamDev-Ltd__OpenFaces,2979,2859,0.9660,0.96900,0.967,0.96900,0.93800,0.4940


### Test

Uses 80% of the data to train the model and the remaining 20% of the data to test it. 
The accuracy is calculated based on the 20% of the data that the model has never seen during training.

In [10]:
results_test = classifier_utils.ProjectsResults(model, projects, non_features_columns, training=False)

In [11]:
report_test = results_test.get_report_df(include_overall=True)
report_test

,project,observations,observations (wt NaN),precision,recall,f1-score,accuracy,baseline (majority),improvement
0,CCI-MIT__XCoLab,5512,3757,0.979,0.9810,0.97900,0.9810,0.57300,0.9560
1,apache__directory-server,845,652,0.922,0.9220,0.92100,0.9220,0.53300,0.8330
2,jgralab__jgralab,2072,1802,0.879,0.8840,0.88000,0.8840,0.50400,0.7650
3,CloudStack-extras__CloudStack-archive,1424,1106,0.833,0.8220,0.82400,0.8220,0.42800,0.6890
4,Unidata__thredds,1154,950,0.903,0.9090,0.90500,0.9090,0.72200,0.6720
5,Ramblurr__Anki-Android,892,759,0.793,0.8100,0.79900,0.8100,0.44100,0.6610
6,apache__accumulo,4113,3148,0.859,0.8660,0.85900,0.8660,0.64200,0.6250
7,getrailo__railo,815,572,0.729,0.7300,0.72100,0.7300,0.39300,0.5540
8,zkoss__zk,1087,881,0.785,0.7870,0.78300,0.7870,0.56500,0.5110
9,apache__lucene-solr,1256,974,0.608,0.6110,0.59800,0.6110,0.26700,0.4700


# Summary of the results

In [12]:
df_inner = pd.merge(report, report_test, on='project', how='inner', suffixes=('_cv', '_test'))
df = df_inner.filter(regex=("project|accuracy.*|baseline.*_test|precision_test|recall_test|f1-score_test")).copy()
df = df.rename(columns={"precision_test": "precision", 'recall_test':'recall', 'f1-score_test':'f1-score', 'baseline (majority)_test': 'baseline'})
df['norm._improv.'] = df.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_test'], x['baseline']), axis=1)
columns = ['project', 'accuracy_cv', 'baseline', 'accuracy_test', 'norm._improv.', 'precision', 'recall', 'f1-score']
def fix_project_name(project_name):
    return project_name.replace('__','/')
summary = df[columns]
summary['project'] = summary['project'].apply(fix_project_name)
summary = summary.round(3).dropna()
summary

,project,accuracy_cv,baseline,accuracy_test,norm._improv.,precision,recall,f1-score
0,CCI-MIT/XCoLab,0.976,0.573,0.981,0.956,0.979,0.981,0.979
1,apache/directory-server,0.942,0.533,0.922,0.833,0.922,0.922,0.921
2,jgralab/jgralab,0.867,0.504,0.884,0.766,0.879,0.884,0.880
3,CloudStack-extras/CloudStack-archive,0.806,0.428,0.822,0.689,0.833,0.822,0.824
4,Unidata/thredds,0.918,0.722,0.909,0.673,0.903,0.909,0.905
5,apache/accumulo,0.862,0.642,0.866,0.626,0.859,0.866,0.859
6,Ramblurr/Anki-Android,0.739,0.441,0.810,0.660,0.793,0.810,0.799
7,getrailo/railo,0.710,0.393,0.730,0.555,0.729,0.730,0.721
8,apache/lucene-solr,0.646,0.267,0.611,0.469,0.608,0.611,0.598
9,TeamDev-Ltd/OpenFaces,0.969,0.942,0.968,0.448,0.961,0.968,0.964


In [13]:
summary.to_csv('../../data/results/experiment_results.csv', index=None)

# Results per class

In [14]:
baseline_combination = ConstantClassifier('Combination')
baseline_version1 = ConstantClassifier('Version 1')
baseline_version2 = ConstantClassifier('Version 2')
baseline_concatenationv1v2 = ConstantClassifier('ConcatenationV1V2')
baseline_concatenationv2v1 = ConstantClassifier('ConcatenationV2V1')
baseline_manual = ConstantClassifier('Manual')
baseline_none = ConstantClassifier('None')
model = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=100, max_features=0.3, min_samples_leaf=1)
models = [baseline_combination, baseline_version1, baseline_version2,
          baseline_concatenationv1v2, baseline_concatenationv2v1, baseline_manual,
          baseline_none, model]
all_models_names = ['baseline_combination', 'baseline_version1',
                'baseline_version2', 'baseline_concatenationv1v2',
                'baseline_concatenationv2v1', 'baseline_manual', 'baseline_none',
                'random forest']

In [15]:
models_results = {}
for model, model_name in zip(models, all_models_names):
    models_results[model_name] = classifier_utils.ProjectsResults(model, projects, non_features_columns, training=False)

## Combination

In [16]:
models_names = ['baseline_combination', 'random forest']
combination = classifier_utils.compare_models_per_class('Combination', models_results, models_names)
combination['target_class'] = 'Combination'
combination

,model,precision,recall,f1-score,target_class
0,baseline_combination,0.100855,1.000000,0.170993,Combination
1,random forest,0.574964,0.553709,0.545107,Combination


## Version 1

In [17]:
models_names = ['baseline_version1', 'random forest']
version1 = classifier_utils.compare_models_per_class('Version 1', models_results, models_names)
version1['target_class'] = 'Version 1'
version1

,model,precision,recall,f1-score,target_class
0,baseline_version1,0.519473,1.000000,0.649132,Version 1
1,random forest,0.847366,0.874226,0.859793,Version 1


## Version 2

In [18]:
models_names = ['baseline_version2', 'random forest']
version2 = classifier_utils.compare_models_per_class('Version 2', models_results, models_names)
version2['target_class'] = 'Version 2'
version2

,model,precision,recall,f1-score,target_class
0,baseline_version2,0.237367,1.000000,0.351846,Version 2
1,random forest,0.740505,0.733832,0.734320,Version 2


## ConcatenationV1V2

In [19]:
models_names = ['baseline_concatenationv1v2', 'random forest']
concatv1v2 = classifier_utils.compare_models_per_class('ConcatenationV1V2', models_results, models_names)
concatv1v2['target_class'] = 'ConcatenationV1V2'
concatv1v2

,model,precision,recall,f1-score,target_class
0,baseline_concatenationv1v2,0.041066,1.000000,0.075335,ConcatenationV1V2
1,random forest,0.793799,0.545609,0.632351,ConcatenationV1V2


## ConcatenationV2V1

In [20]:
models_names = ['baseline_concatenationv2v1', 'random forest']
concatv2v1 = classifier_utils.compare_models_per_class('ConcatenationV2V1', models_results, models_names)
concatv2v1['target_class'] = 'ConcatenationV2V1'
concatv2v1

,model,precision,recall,f1-score,target_class
0,baseline_concatenationv2v1,0.008445,1.000000,0.016695,ConcatenationV2V1
1,random forest,0.666667,0.446429,0.495238,ConcatenationV2V1


## Manual

In [21]:
models_names = ['baseline_manual', 'random forest']
manual = classifier_utils.compare_models_per_class('Manual', models_results, models_names)
manual['target_class'] = 'Manual'
manual

,model,precision,recall,f1-score,target_class
0,baseline_manual,0.118856,1.000000,0.204728,Manual
1,random forest,0.626303,0.556366,0.580444,Manual


## None

In [22]:
models_results['random forest'].get_class_score_df('None', 'precision')

,project,None
0,Ramblurr__Anki-Android,NaN
1,apache__directory-server,NaN
2,android__platform_frameworks_base,NaN
3,freenet__fred,NaN
4,alexo__wro4j,NaN
5,apache__lucene-solr,1.0
6,getrailo__railo,NaN
7,atlasapi__atlas,NaN
8,hibernate__hibernate-orm,NaN
9,CloudStack-extras__CloudStack-archive,NaN


In [23]:
models_names = ['baseline_none', 'random forest']
none = classifier_utils.compare_models_per_class('None', models_results, models_names)
none['target_class'] = 'None'
none

,model,precision,recall,f1-score,target_class
0,baseline_none,0.009719,1.000,0.019112,None
1,random forest,1.000000,0.125,0.222222,None


Not enough "None" instances in the projects.

In [24]:
summary_classes = pd.concat([combination, version1, version2, concatv1v2, concatv2v1, manual, none])
summary_classes = summary_classes[['target_class', 'model', 'precision', 'recall', 'f1-score']]
display(summary_classes)
summary_classes.to_csv('../../data/results/experiment_by_class.csv', index=False)

,target_class,model,precision,recall,f1-score
0,Combination,baseline_combination,0.100855,1.000000,0.170993
1,Combination,random forest,0.574964,0.553709,0.545107
0,Version 1,baseline_version1,0.519473,1.000000,0.649132
1,Version 1,random forest,0.847366,0.874226,0.859793
0,Version 2,baseline_version2,0.237367,1.000000,0.351846
1,Version 2,random forest,0.740505,0.733832,0.734320
0,ConcatenationV1V2,baseline_concatenationv1v2,0.041066,1.000000,0.075335
1,ConcatenationV1V2,random forest,0.793799,0.545609,0.632351
0,ConcatenationV2V1,baseline_concatenationv2v1,0.008445,1.000000,0.016695
1,ConcatenationV2V1,random forest,0.666667,0.446429,0.495238
